In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder , StandardScaler

df=pd.read_csv(r"C:\Users\DHARMESH M\Documents\Projects\SIG-COG\Dataset\features_dataset.csv")

features=['SNR_dB','BER','PacketLoss_pct','Jitter_ms','Throughput_Mbps','SNR_trend','Jitter_spike']
y_label=df['Label'].values
X_raw=df[features].values

scalar = StandardScaler()
X = scalar.fit_transform(X_raw)

label_encoder = LabelEncoder()
label_encoder.fit(y_label)  # First, fit it

print("Class order:", label_encoder.classes_)  # Now this works

y = label_encoder.transform(y_label)


window_size=15
x_seq=[]
y_seq=[]

for i in range(len(X)-window_size+1):
    window= X[i:i+window_size]
    label = y[i+window_size-1]
    x_seq.append(window)
    y_seq.append(label)

X_seq= np.array(x_seq)
y_seq = np.array(y_seq)
print("Shape of X_seq:", X_seq.shape)
print("Shape of y_seq:", y_seq.shape)

Class order: ['Good' 'Moderate' 'Poor']
Shape of X_seq: (14986, 15, 7)
Shape of y_seq: (14986,)


In [20]:
import sklearn.model_selection as model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

In [7]:
pip install tensorflow.keras


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

num_classes = len(np.unique(y_seq))

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(window_size, X_seq.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=15,
    batch_size=64,
    verbose=1
)

model.save("link_state_lstm.h5")

Epoch 1/15


c:\Users\DHARMESH M\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.6434 - loss: 0.7707 - val_accuracy: 0.9733 - val_loss: 0.1186
Epoch 2/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9712 - loss: 0.1045 - val_accuracy: 0.9797 - val_loss: 0.0570
Epoch 3/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9781 - loss: 0.0640 - val_accuracy: 0.9790 - val_loss: 0.0499
Epoch 4/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - accuracy: 0.9800 - loss: 0.0530 - val_accuracy: 0.9843 - val_loss: 0.0401
Epoch 5/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.9824 - loss: 0.0456 - val_accuracy: 0.9850 - val_loss: 0.0410
Epoch 6/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.9816 - loss: 0.0479 - val_accuracy: 0.9843 - val_loss: 0.0403
Epoch 7/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.9833 - loss: 0.0444 - val_accuracy: 0.9843 - val_loss: 0.0377
Epoch 8/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - accuracy: 0.9825 - loss: 0.0462 - val_accuracy: 0.98

In [22]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc*100:.2f}%")


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9804 - loss: 0.0436
Test Accuracy: 98.23%
